## Import Library

In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd

### Load GeoJson Files

In [6]:
grid = gpd.read_file("data/area/grid.geojson")
grid = grid.set_index("id")
grid.head()

,grid_area,geometry
id,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926..."
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926..."
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926..."
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926..."
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926..."


In [19]:
grid.crs

<Projected CRS: EPSG:26711>
Name: NAD27 / UTM zone 11N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 120°W and 114°W - onshore. Canada - Alberta; British Columbia; Northwest Territories; Nunavut. Mexico. United States (USA) - California; Idaho; Nevada; Oregon; Washington.
- bounds: (-120.0, 26.93, -114.0, 78.13)
Coordinate Operation:
- name: UTM zone 11N
- method: Transverse Mercator
Datum: North American Datum 1927
- Ellipsoid: Clarke 1866
- Prime Meridian: Greenwich

### Function to create buffer + calcuate area of green space within buffer

In [11]:
#set radius of buffer in here
RADIUS_BUFFER = 500

#Function to create centriod and buffer 

def centriod_buffer(df):
    df["centriod"]=df.geometry.centroid
    df["buffer"]=df.geometry.buffer(RADIUS_BUFFER)

#function to calculate green space ratio

def green_percentage(df, numerator_col, denominator_col="grid_area"):
    def get_buffer_percentage(buffer):
        intersects = df["geometry"].intersects(buffer)
        denominator = df[intersects][denominator_col].sum()
        numerator = df[intersects][numerator_col].sum()
        return numerator/denominator*100

    buffer_series = df["buffer"]
    return buffer_series.map(get_buffer_percentage)

### Calculate Area + Clean Datatable

In [7]:
def area_percent (df, n_col, d_col, name): 
    df[name] = df[n_col]/df[d_col]*100
    return df

Designated Green Space

In [50]:
dgs_inter = gpd.read_file("data/area/dgs_intersect.geojson")
dgs_inter.head(1)

,id,grid_id,grid_grid_area,dsg_area,geometry
0,way/168759911,1932,10004,3127,"MULTIPOLYGON (((-114.49886 62.47056, -114.4989..."


In [51]:
#Designated Green Space Area Percentage
dgs_area = area_percent(dgs_inter, "dsg_area", 'grid_grid_area', "dgs_percent")
dgs_area.head(1)

,id,grid_id,grid_grid_area,dsg_area,geometry,dgs_percent
0,way/168759911,1932,10004,3127,"MULTIPOLYGON (((-114.49886 62.47056, -114.4989...",31.257497


In [52]:
dgs_area = dgs_area.drop(columns=["id", "grid_grid_area", "geometry"]).rename(columns={"dsg_area":"dgs_area"})
dgs_area = dgs_area.set_index("grid_id")
dgs_area.head()

,dgs_area,dgs_percent
grid_id,,
1932,3127,31.257497
1933,2473,24.720112
2085,1425,14.244302
2086,9405,94.012395
2087,8265,82.616953


Quasi Green Space

In [27]:
qgs_inter = gpd.read_file("data/area/qgs_intersect.geojson")
qgs_inter.head(1)

,id,grid_id,grid_grid_area,qsg_area,geometry
0,way/164229239,2853,10004,368,"MULTIPOLYGON (((-114.48740 62.47256, -114.4870..."


In [28]:
#Quasi Green Space Area Percentage
qgs_area = area_percent(qgs_inter, "qsg_area", 'grid_grid_area', "qgs_percent")
qgs_area.head(1)

,id,grid_id,grid_grid_area,qsg_area,geometry,qgs_percent
0,way/164229239,2853,10004,368,"MULTIPOLYGON (((-114.48740 62.47256, -114.4870...",3.678529


In [29]:
qgs_area = qgs_area.drop(columns=["id", "geometry","grid_grid_area"]).rename(columns={"qsg_area":"qgs_area"})
qgs_area = qgs_area.set_index("grid_id")
qgs_area.head()

,qgs_area,qgs_percent
grid_id,,
2853,368,3.678529
2854,19,0.189924
3006,884,8.836465
3007,8322,83.186725
3008,5181,51.789284


Wild Green Space

In [30]:
wild_inter = gpd.read_file("data/area/wild_intersect.geojson")
wild_inter.head(1)

,id,gird_id,gird_grid_area,wild_area,geometry
0,way/939527896,366,8987,1698,"MULTIPOLYGON (((-114.51656 62.49405, -114.5162..."


In [31]:
#Wild Green Space Area Percentage
wild_area = area_percent(wild_inter, "wild_area", "gird_grid_area", "wild_percent")
wild_area.head(1)

,id,gird_id,gird_grid_area,wild_area,geometry,wild_percent
0,way/939527896,366,8987,1698,"MULTIPOLYGON (((-114.51656 62.49405, -114.5162...",18.893958


In [32]:
wild_area = wild_area.drop(columns=["id","geometry",'gird_grid_area']).rename(columns={"gird_id":"grid_id"})
wild_area = wild_area.set_index("grid_id")
wild_area.head()

,wild_area,wild_percent
grid_id,,
366,1698,18.893958
367,7559,87.813662
368,5560,67.557716
369,356,4.533877
370,3199,42.801713


Water Green Space

In [33]:
water_inter = gpd.read_file("data/area/water_intersect.geojson")
water_inter.head(1)

,id,id_2,grid_area,water_area,geometry
0,relation/1475714,370,7474,1646,"MULTIPOLYGON (((-114.51729 62.49009, -114.5175..."


In [34]:
#Water Green Space Area Percentage
water_area = area_percent(water_inter, "water_area", "grid_area", "water_percent")
water_area.head(1)

,id,id_2,grid_area,water_area,geometry,water_percent
0,relation/1475714,370,7474,1646,"MULTIPOLYGON (((-114.51729 62.49009, -114.5175...",22.023013


In [35]:
water_area = water_area.drop(columns=["id","geometry", "grid_area"]).rename(columns={"id_2":"grid_id"})
water_area = water_area.set_index("grid_id")
water_area.head()

,water_area,water_percent
grid_id,,
370,1646,22.023013
371,3071,43.277903
372,2907,43.271807
373,378,5.962145
375,4364,78.165861


Total Green Space

In [4]:
total_inter = gpd.read_file("data/area/total_intersect.geojson")
total_inter.head(1)

,id,id_2,grid_area,totalgs_area,geometry
0,way/168759911,366,8987,1698,"MULTIPOLYGON (((-114.51656 62.49405, -114.5162..."


In [37]:
total_area = area_percent(total_inter, "totalgs_area", "grid_area", "totalgs_percent")
total_area.head(1)

,id,id_2,grid_area,totalgs_area,geometry,totalgs_percent
0,way/168759911,366,8987,1698,"MULTIPOLYGON (((-114.51656 62.49405, -114.5162...",18.893958


In [38]:
total_area = total_area.drop(columns=["id", "geometry","grid_area"]).rename(columns={"id_2":"grid_id"})
total_area = total_area.set_index("grid_id")
total_area.head()

,totalgs_area,totalgs_percent
grid_id,,
366,1698,18.893958
367,7559,87.813662
368,5560,67.557716
369,356,4.533877
370,4191,56.074391


### Merge all table to one

In [43]:
join1 = grid.join(dgs_area, how="left")
join2 = join1.join(qgs_area, how="left")
join3 = join2.join(wild_area, how="left")
join4 = join3.join(water_area, how="left")
final = join4.join(total_area, how="left")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent
id,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Functions to Calculate Green Space Percentage over a Buffer

In [44]:
#set radius of buffer in here
RADIUS_BUFFER = 500

In [45]:
#Function to create centriod and buffer 

def centriod_buffer(df):
    df["centriod"]=df.geometry.centroid
    df["buffer"]=df.geometry.buffer(RADIUS_BUFFER)

In [46]:
#function to calculate green space ratio

def green_percentage(df, numerator_col, denominator_col="grid_area"):
    def get_buffer_percentage(buffer):
        intersects = df["geometry"].intersects(buffer)
        denominator = df[intersects][denominator_col].sum()
        numerator = df[intersects][numerator_col].sum()
        return numerator/denominator*100

    buffer_series = df["buffer"]
    return buffer_series.map(get_buffer_percentage)


In [47]:
final.crs

<Projected CRS: EPSG:26711>
Name: NAD27 / UTM zone 11N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 120°W and 114°W - onshore. Canada - Alberta; British Columbia; Northwest Territories; Nunavut. Mexico. United States (USA) - California; Idaho; Nevada; Oregon; Washington.
- bounds: (-120.0, 26.93, -114.0, 78.13)
Coordinate Operation:
- name: UTM zone 11N
- method: Transverse Mercator
Datum: North American Datum 1927
- Ellipsoid: Clarke 1866
- Prime Meridian: Greenwich

### Calculation of smoothed percentage for each category

In [50]:
#Designated Green Space
centriod_buffer(final)
final["dgs_percentage"] = green_percentage(final, "dgs_area")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,centriod,buffer,dgs_percentage
id,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0


In [55]:
#Quasi Green Space
centriod_buffer(final)
final["qgs_percentage"] = green_percentage(final, "qgs_area")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,centriod,buffer,dgs_percentage,qgs_percentage
id,,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0,0.0
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0,0.0
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0,0.0
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0,0.0
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0,0.0


In [58]:
#Wild Green Space
centriod_buffer(final)
final["wild_percentage"] = green_percentage(final, "wild_area")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,centriod,buffer,dgs_percentage,qgs_percentage,wild_percentage
id,,,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0,0.0,8.959444
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0,0.0,12.194682
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0,0.0,16.586166
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0,0.0,18.438794
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0,0.0,18.910850


In [60]:
#Water Green Space
centriod_buffer(final)
final["water_percentage"] = green_percentage(final, "water_area")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,centriod,buffer,dgs_percentage,qgs_percentage,wild_percentage,water_percentage
id,,,,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0,0.0,8.959444,3.818492
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0,0.0,12.194682,9.699392
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0,0.0,16.586166,11.018301
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0,0.0,18.438794,11.170966
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0,0.0,18.910850,9.006611


In [62]:
#Total Green Space
centriod_buffer(final)
final["total_greenspace_percentage"] = green_percentage(final, "totalgs_area")
final.head()

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,centriod,buffer,dgs_percentage,qgs_percentage,wild_percentage,water_percentage,total_greenspace_percentage
id,,,,,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0,0.0,8.959444,3.818492,12.665663
206,1251,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0,0.0,12.194682,9.699392,19.451534
207,873,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0,0.0,16.586166,11.018301,24.794778
208,495,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0,0.0,18.438794,11.170966,27.133533
209,124,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0,0.0,18.910850,9.006611,25.638572


In [64]:
#Export final table as GeoJson

import fiona
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [66]:
final = final.drop(columns=["centriod", "buffer"])
final.head(1)

,grid_area,geometry,dgs_area,dgs_percent,qgs_area,qgs_percent,wild_area,wild_percent,water_area,water_percent,totalgs_area,totalgs_percent,dgs_percentage,qgs_percentage,wild_percentage,water_percentage,total_greenspace_percentage
id,,,,,,,,,,,,,,,,,
205,146,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,8.959444,3.818492,12.665663


In [67]:
final.to_file("data/final/greenspace.geojson", driver='GeoJSON')